In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.basemap import Basemap
import os
import unicodedata
import re
import imageio

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
def slugify(value):
    """
    https://stackoverflow.com/a/295466
    Normalizes string, converts to lowercase, removes non-alpha characters,
    and converts spaces to hyphens.
    """
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore')
    value = re.sub('[^\w\s-]', '', str(value)).strip().lower()
    value = re.sub('[-\s]+', '-', str(value))
    
    return value.strip('b')

In [ ]:
data = pd.read_csv('./scratch/export.csv', encoding='windows-1250', sep=';')

In [ ]:
#dropna
data.dropna(how='all', subset=['Lat', 'ObsLat'], inplace=True)
data.dropna(how='all', subset=['TaxonNameCZ'], inplace=True)

In [ ]:
#omezeni na aktulalni dny
#data = data[(data.Week > 28) & (data.Week < 32)]

In [ ]:
ordr = 0
for name in data.TaxonNameCZ.value_counts().keys():
    ordr += 1
    print(name)
    ser = data[data.TaxonNameCZ == name]
    for i in range(1, 13):
        se = ser[ser.Month == i]
        plt.figure(ordr + i)
        plt.figure(figsize=(10, 8))
        map = Basemap(resolution='c', # c, l, i, h, f or None
                projection='merc',
                lat_0=50.0835494, lon_0=14.4341414,
                llcrnrlon=14.224453, llcrnrlat=49.941936, urcrnrlon=14.706795, urcrnrlat=50.177403)

        map.readshapefile('./scratch/mc/TMMESTSKECASTI_P', 'mc')

        x, y = map(se.Lon.values, se.Lat.values)

        map.scatter(x, y, 2, marker='o', color='r')
        plt.title(name + ' - ' + str(i) + '. měsíc')
        plt.savefig('./scratch/img/' + str(i) + slugify(name) + '.png', bbox_inches='tight')
        plt.close()
    
    #make gif
    images = []
    for filename in os.listdir('./scratch/img/'):
        images.append(imageio.imread('./scratch/img/' + filename))
    imageio.mimsave('./gifs/' + slugify(name) + '.gif', images,  duration=1)
    
    #clean
    for filename in os.listdir('./scratch/img/'):
        os.remove('./scratch/img/' + filename)

In [ ]:
#make html prez
with open('animated-maps.html', 'w', encoding='utf-8') as f:
    f.write('<!doctype html><head><meta charset="utf-8"></head><body>')
    f.write('vyrobil <a target="_blank" href="https://datastory.cz">datastory</a>'
    + ', <a target="_blank" href="https://github.com/datastory/avif-stats">kód volný</a>, použitá data (roky 2013-16) pochází'
    + ' z <a target="_blank" href="http://www.birds.cz/avif/">faunistické databáze AVIF</a><br><br>')
    f.write('<b>na stránce můžete hledat kombinací kláves Ctrl + F</b><br>')
    for name in data.TaxonNameCZ.value_counts().keys():
        f.write('<h3>' + name + '</h3><br><img src="./gifs/' + slugify(name) + '.gif"><br>')
    f.write('</body></html>')